In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

In [28]:
def save_metrics(lmd_cont_val):
    runs = api.runs("mukundshankar/tmr_with_dtw", filters={"config.lmd_contrastive":lmd_cont_val})

    summary_list, config_list, name_list = [], [], []
    for run in runs: 
        summary_list.append(run.summary._json_dict)

        config_list.append(
            {k: v for k,v in run.config.items()
            if not k.startswith('_')})

        name_list.append(run.name)

    runs_df = pd.DataFrame({
        "summary": summary_list,
        "config": config_list,
        "name": name_list
        })
    
    new_df_dict = {}
    for idx, row in runs_df.iterrows():
        lmd_dtw_val = row['config']['lmd_dtw']
        normal_metrics = row['summary']['normal']
        threshold_metrics = row['summary']['threshold']
        nsim_metrics = row['summary']['nsim']
        guo_metrics = row['summary']['guo']

        t2m_keys = ['t2m/R01', 't2m/R02', 't2m/R03', 't2m/R05', 't2m/R10', 't2m/MedR']
        m2t_keys = ['m2t/R01', 'm2t/R02', 'm2t/R03', 'm2t/R05', 'm2t/R10', 'm2t/MedR']
        
        normal_t2m_metrics = {k: normal_metrics[k] for k in t2m_keys}
        normal_m2t_metrics = {k: normal_metrics[k] for k in m2t_keys}
        threshold_t2m_metrics = {k: threshold_metrics[k] for k in t2m_keys}
        threshold_m2t_metrics = {k: threshold_metrics[k] for k in m2t_keys}
        nsim_t2m_metrics = {k: nsim_metrics[k] for k in t2m_keys}
        nsim_m2t_metrics = {k: nsim_metrics[k] for k in m2t_keys}
        guo_t2m_metrics = {k: guo_metrics[k] for k in t2m_keys}
        guo_m2t_metrics = {k: guo_metrics[k] for k in m2t_keys}

        dict_to_add = {
            "normal_t2m": normal_t2m_metrics,
            "normal_m2t": normal_m2t_metrics,
            "threshold_t2m": threshold_t2m_metrics,
            "threshold_m2t": threshold_m2t_metrics,
            "nsim_t2m": nsim_t2m_metrics,
            "nsim_m2t": nsim_m2t_metrics,
            "guo_t2m": guo_t2m_metrics,
            "guo_m2t": guo_m2t_metrics
        }
        new_df_dict[str(lmd_dtw_val)] = dict_to_add

    data = new_df_dict

    # Prepare a list for DataFrame construction
    records = []
    samples = ['0.1', '0.25', '0.5', '0.75', '0.9']
    metrics = ['R01', 'R02', 'R03', 'R05', 'R10', 'MedR']

    # Process the data for DataFrame conversion
    for sample in samples:
        for category_key, category_data in data[sample].items():
            category = category_key.split('_')[0]
            metric_prefix = category_key.split('_')[1]  # 't2m' or 'm2t'
            for metric in metrics:
                metric_key = f"{metric_prefix}/{metric}"
                value = category_data.get(metric_key, None)  # Handle missing data gracefully
                metric_label = f"{metric_prefix}_{metric}"
                records.append((category, metric_label, sample, value))

    # Create the DataFrame
    df = pd.DataFrame(records, columns=['Category', 'Metric', 'Sample', 'Value'])
    df_pivot = df.pivot_table(index='Metric', columns=['Category', 'Sample'], values='Value')

    # Split the DataFrame into four based on the category
    df_normal = df_pivot.xs('normal', level='Category', axis=1)
    df_threshold = df_pivot.xs('threshold', level='Category', axis=1)
    df_nsim = df_pivot.xs('nsim', level='Category', axis=1)
    df_guo = df_pivot.xs('guo', level='Category', axis=1)

    df_normal.to_csv(f'comparitive_metrics/cont_{lmd_cont_val}_normal.csv')
    df_threshold.to_csv(f'comparitive_metrics/cont_{lmd_cont_val}_threshold.csv')
    df_nsim.to_csv(f'comparitive_metrics/cont_{lmd_cont_val}_nsim.csv')
    df_guo.to_csv(f'comparitive_metrics/cont_{lmd_cont_val}_guo.csv')

In [29]:
lmd_vals = [0.1, 0.25, 0.5, 0.75, 0.9]
for val in lmd_vals:
    save_metrics(val)